In [23]:
import pandas as pd
import numpy as np

In [24]:
pd.set_option('display.max_columns', 60)
myData=pd.read_csv('imputedData.csv')
myData.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,False,False
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,False,False
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,4,1,9,3,Returning_Visitor,False,False
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,Feb,3,2,2,4,Returning_Visitor,False,False
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,Feb,3,3,1,4,Returning_Visitor,True,False


In [25]:
#One hot encoding of categorical variables

#Create list with features to be dummified cols.
nonum_feats_names = ['Month','OperatingSystems','Browser','Region','VisitorType','Weekend']


dataModel = pd.concat([myData[['Administrative', 'Administrative_Duration', 'Informational','Informational_Duration', 
                              'ProductRelated','ProductRelated_Duration','BounceRates','ExitRates','PageValues',
                               'TrafficType','SpecialDay']],
                       pd.get_dummies(myData[nonum_feats_names].astype('category')),myData['Revenue']],axis=1)
                      
dataModel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12330 entries, 0 to 12329
Data columns (total 57 columns):
Administrative                   12330 non-null int64
Administrative_Duration          12330 non-null float64
Informational                    12330 non-null int64
Informational_Duration           12330 non-null float64
ProductRelated                   12330 non-null int64
ProductRelated_Duration          12330 non-null float64
BounceRates                      12330 non-null float64
ExitRates                        12330 non-null float64
PageValues                       12330 non-null float64
TrafficType                      12330 non-null int64
SpecialDay                       12330 non-null float64
Month_Aug                        12330 non-null uint8
Month_Dec                        12330 non-null uint8
Month_Feb                        12330 non-null uint8
Month_Jul                        12330 non-null uint8
Month_June                       12330 non-null uint8
Month_Mar    

In [26]:
#Label Encoding of revenue

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
myData['Revenue'] = le.fit_transform(myData['Revenue'])
myData['Revenue'].value_counts()

myData['Revenue'].head()

# getting dependent and independent variables

x = dataModel
# removing the target column revenue from x
x = x.drop(['Revenue'], axis = 1)

y = myData['Revenue']

# checking the shapes
print("Shape of x:", x.shape)
print("Shape of y:", y.shape)

Shape of x: (12330, 56)
Shape of y: (12330,)


In [27]:
# splitting the data

from sklearn.model_selection import train_test_split

x_baseTrain, x_baseTest, y_baseTrain, y_baseTest = train_test_split(x, y, test_size = 0.3, random_state = 42)

# checking the shapes

print("Shape of x_train :", x_baseTrain.shape)
print("Shape of y_train :", y_baseTrain.shape)
print("Shape of x_test :", x_baseTest.shape)
print("Shape of y_test :", y_baseTest.shape)

Shape of x_train : (8631, 56)
Shape of y_train : (8631,)
Shape of x_test : (3699, 56)
Shape of y_test : (3699,)


In [28]:
# remove constant features within the same column
constant_features = [
    feat for feat in x_baseTrain.columns if x_baseTrain[feat].std() == 0
]
constant_features
#x_baseTrain.drop(labels=constant_features, axis=1, inplace=True)
#x_baseTest.drop(labels=constant_features, axis=1, inplace=True)

#x_baseTrain.shape, x_baseTest.shape

[]

In [31]:
from sklearn.feature_selection import VarianceThreshold
# remove quasi-constant features
sel = VarianceThreshold(
    threshold=0.01)  # 0.1 indicates 99% of observations approximately

sel.fit(x_baseTrain)  # fit finds the features with low variance

sum(sel.get_support()) # how many not quasi-constant?
features_to_keep = x_baseTrain.columns[sel.get_support()]
print(features_to_keep)

features_to_remove = x_baseTrain.columns[~x_baseTrain.columns.isin(features_to_keep)]
print(features_to_remove)



Index(['Administrative', 'Administrative_Duration', 'Informational',
       'Informational_Duration', 'ProductRelated', 'ProductRelated_Duration',
       'PageValues', 'TrafficType', 'SpecialDay', 'Month_Aug', 'Month_Dec',
       'Month_Feb', 'Month_Jul', 'Month_June', 'Month_Mar', 'Month_May',
       'Month_Nov', 'Month_Oct', 'Month_Sep', 'OperatingSystems_1',
       'OperatingSystems_2', 'OperatingSystems_3', 'OperatingSystems_4',
       'Browser_1', 'Browser_2', 'Browser_4', 'Browser_5', 'Browser_6',
       'Browser_8', 'Browser_10', 'Region_1', 'Region_2', 'Region_3',
       'Region_4', 'Region_5', 'Region_6', 'Region_7', 'Region_8', 'Region_9',
       'VisitorType_New_Visitor', 'VisitorType_Returning_Visitor',
       'Weekend_False', 'Weekend_True'],
      dtype='object')
Index(['BounceRates', 'ExitRates', 'OperatingSystems_5', 'OperatingSystems_6',
       'OperatingSystems_7', 'OperatingSystems_8', 'Browser_3', 'Browser_7',
       'Browser_9', 'Browser_11', 'Browser_12', 'Browser

In [38]:
qconstant_filter = VarianceThreshold(threshold=0.01)
qconstant_filter.fit(x_baseTrain)
len(x_baseTrain.columns[qconstant_filter.get_support()])
qconstant_columns = [column for column in x_baseTrain.columns
                    if column not in x_baseTrain.columns[qconstant_filter.get_support()]]

print(len(qconstant_columns))



13


In [39]:
for column in qconstant_columns:
    print(column)

BounceRates
ExitRates
OperatingSystems_5
OperatingSystems_6
OperatingSystems_7
OperatingSystems_8
Browser_3
Browser_7
Browser_9
Browser_11
Browser_12
Browser_13
VisitorType_Other


In [32]:
# check for duplicated features in the training set
duplicated_feat = []
for i in range(0, len(x_baseTrain.columns)):
    if i % 10 == 0:  # this helps me understand how the loop is going
        print(i)

    col_1 = x_baseTrain.columns[i]

    for col_2 in x_baseTrain.columns[i + 1:]:
        if x_baseTrain[col_1].equals(x_baseTrain[col_2]):
            duplicated_feat.append(col_2)

0
10
20
30
40
50


In [33]:
# check how many features are duplicated
print(len(set(duplicated_feat)))

1


In [35]:
duplicated_feat

['Browser_11']

In [36]:
# we can go ahead and try to identify which set of features
# are identical

duplicated_feat = []
for i in range(0, len(x_baseTrain.columns)):

    col_1 = x_baseTrain.columns[i]

    for col_2 in x_baseTrain.columns[i + 1:]:

        # if the features are duplicated
        if x_baseTrain[col_1].equals(x_baseTrain[col_2]):

            #print them
            print(col_1)
            print(col_2)
            print()

            # and then append the duplicated one to a
            # list
            duplicated_feat.append(col_2)

OperatingSystems_5
Browser_11



In [37]:
x_baseTrain.to_csv('train.csv')

In [ ]:
# we can then remove the features like this
x_baseTrain = sel.transform(x_baseTrain)
x_baseTest = sel.transform(x_baseTest)

x_baseTrain.shape, x_baseTest.shape

In [ ]:
# sklearn transformations lead to numpy arrays
# Lets transform the arrays back to dataframes
# please be mindful of getting the columns assigned
# correctly

x_baseTrain= pd.DataFrame(x_baseTrain)
x_baseTrain.columns = features_to_keep

x_baseTest= pd.DataFrame(x_baseTest)
x_baseTest.columns = features_to_keep

x_baseTrain.shape, x_baseTest.shape
x_baseTrain